# Erstellen und Trainieren der KI

In [1]:
# Needed imports
import numpy as np
from pathlib import Path
from tensorflow import keras
from datetime import datetime

# Needed variabel
dataset_folder = Path(__file__).parent / "dataset"
sequence_length = 64
model_path = Path(__file__).parent / "model.h5"

In [17]:
# Load the dataset
songs = np.load(dataset_folder / "songs.npy")

print(songs.shape)
print(songs[:10])

songs = songs.tolist()

(2576,)
[ 5 16  9 16 16 16 10 16 11 16]


## Erstellen der Trainingsdaten
Wir erstellen sequenzen von eienr bestimmten länge und sagen dem Netzwerk, ok was kommt als nächstes. Dies macht man so lange, bis man alle Daten durchhat. Das schafft man, indem man die die Trainignsdaten nimmt, ein Teil herausschneided und dan den Ausschnit zum Herusschneiden immer eins weiter nach rechts shifted, bis man alle Daten durch ist.

```bash
# Data
[12, 13, 14, 15, 16, 17, ...]

# Erster input und targed
[12, 13] -> 14

#Zweiter input und targed
[13, 14] -> 15
```

In [19]:
# 100 symbols, 64 sl
# 100 - 64 = 36
num_sequences = len(songs) - sequence_length

inputs = []
targets = []

for i in range(num_sequences):
    inputs.append(songs[i:i+sequence_length])
    targets.append(songs[i+sequence_length])

In [20]:
print(f"Num sequences: {len(inputs)}")
print("the first input sequence:")
print(inputs[0])
print("the first target sequence:")
print(targets[0])
print()
print("The whole dataset for the first input sequence:")
print(songs[:sequence_length+1])
print()
print(len(inputs), len(targets))

Num sequences: 2512
the first input sequence:
[5, 16, 9, 16, 16, 16, 10, 16, 11, 16, 16, 10, 9, 16, 13, 16, 16, 16, 16, 16, 10, 16, 16, 16, 12, 16, 11, 16, 16, 10, 9, 16, 8, 16, 16, 16, 9, 16, 10, 16, 16, 16, 16, 16, 17, 16, 16, 16, 5, 16, 13, 16, 16, 16, 13, 16, 11, 16, 16, 10, 9, 16, 15, 16]
the first target sequence:
16

The whole dataset for the first input sequence:
[5, 16, 9, 16, 16, 16, 10, 16, 11, 16, 16, 10, 9, 16, 13, 16, 16, 16, 16, 16, 10, 16, 16, 16, 12, 16, 11, 16, 16, 10, 9, 16, 8, 16, 16, 16, 9, 16, 10, 16, 16, 16, 16, 16, 17, 16, 16, 16, 5, 16, 13, 16, 16, 16, 13, 16, 11, 16, 16, 10, 9, 16, 15, 16, 16]

2512 2512


## One-Hot encoding
Jeder mögliche Wert den es geben kann wird als Reihe in einer tabelle gesehen.
![](./Bilder/oenHot.png)

In unserem Fall ist ist die größe der Tabelle gleich der Anzahl an Elemente in unserem `mapping.json`. In unserem Fall sollte das 18 sein.

In [21]:
print(len(inputs))

2512


In [22]:
vacabulay_size = len(np.unique(songs))
inputs = keras.utils.to_categorical(inputs, num_classes=vacabulay_size)
targets = np.array(targets)

In [23]:
print(f"Vocabulary size: {vacabulay_size}")
print(inputs.shape)
print()
print(inputs)

Vocabulary size: 18
(2512, 64, 18)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]]

 ...

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 1. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  

In [24]:
print(targets.shape)
print()
print(targets)

(2512,)

[16 16 16 ...  0  0  0]


## Erstellen des Modells


In [10]:
output_layer_size = vacabulay_size  # Number of unique symbols in the dataset
hidden_layer_sizes = [256, 256]      # Number of neurons in each hidden layer
loss = "sparse_categorical_crossentropy"    # Loss function
optimizer = "adam"                          # Optimizer
activation = "softmax"                      # Activation function
learning_rate = 0.001                       # Learning rate

Hier erstellt man nun das konkrete Netzwerk.

`shape = (None, output_layer_size)`
1. Variable `None`-> Wie viele Zeitstemnpel hat man. Bei None wird das automatisch ermittelt 
2. Variable `output_layer_size` -> Wie viele Inputs hat man pro Eingabe. Dies ist hier die Anzahl der verwendeten Noten

Dem layer wird dann hidden LSTM layer hinzugefügt und ein Dropout, welches gegen Overfitting hilft.

In [11]:
# create the model achitecture
# model = keras.Sequential() # <- Would be a good idea to use this

# Input layer
input_layer = keras.layers.Input(shape=(None, output_layer_size))

# Hidden layers
x = input_layer
for hidden_layer_size in hidden_layer_sizes:
    x = keras.layers.LSTM(hidden_layer_size, return_sequences=True)(x)

# Avoid overfitting
x = keras.layers.Dropout(0.3)(x)

# Output layer. Full connection -> Dense
output_layer = keras.layers.Dense(output_layer_size, activation=activation)(x)

# The actual model
model = keras.Model(input_layer, output_layer)

In [12]:
# Compile the model
model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 18)]        0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 256)         281600    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
dropout (Dropout)            (None, None, 256)         0         
_________________________________________________________________
dense (Dense)                (None, None, 18)          4626      
Total params: 811,538
Trainable params: 811,538
Non-trainable params: 0
_________________________________________________________________


## Trainiere das Modell

In [14]:
print(inputs.shape)
print(targets.shape)

(2512, 64, 18)
(2512,)


In [15]:
epochs = 50
batch_size = 64  # TODO Equal to sequence_length ?

model.fit(inputs, targets, epochs=epochs)

Epoch 1/50


InvalidArgumentError:  assertion failed: [Condition x == y did not hold element-wise:] [x (sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/Shape_1:0) = ] [32 1] [y (sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/strided_slice:0) = ] [32 64]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert (defined at C:\Users\Florian Glaser\AppData\Local\Temp\ipykernel_20936\1927455679.py:4) ]] [Op:__inference_train_function_5586]

Function call stack:
train_function


## Speicher das Modell

In [ ]:
model.save(model_path)